# Discrete Optimization : Peg Solitaire Solver 

**Author: KERROUMI Mohamed**

**Professor: TALBOT Hugues**

June 2019

**CentraleSupélec**

## 1 - Introduction
Peg Solitaire is a game played on a board of a given shape consisting of a number of holes. In any given configuration, each hole contains at most one peg (In the picture below, white circle presents a hole without a peg , and black circle prensents a hole with a peg). A peg solitaire has two given configurations (starting configuration and the finishing configuration), The goal is to start from the starting configuration and remove all pegs but one, by jumping pegs from one side of an occupied peg hole to an empty space, removing the peg which was jumped over, and finally reach the finishing configuration.
<img src="img/1.png" alt="Drawing" style="width: 500px;"/>

## 2 - Integer Programming
**This section has been extracted and detailed from an original paper [1]**

We assume that all the holes on a given board are indexed by integer numbers {1, 2, ... ,n} . The board of Figure 1 has 33 holes, which implies n = 33. We describe a state of certain configuration (pegs in the holes) by a binary vector p of length n
satisfying that the i-th element of p is 1 if and only if the hole i contains a peg. We can take for example the order of i while enumerating holes horizontally (see Figure 2). We denote the starting configuration by ps and the finishing configuration by pf. Therefore we have:
$$ps = [1 . . .  1  0  1 . . . 1]$$
$$pf = [0 . . .  0  1  0 . . . 0]$$
Let J be the family of all the sequences of consecutive three holes on a given board.
Each element in J corresponds to a certain jump and so we can denote a jump by a unit
vector indexed by J. In the rest of this paper, we assume that all the elements in J are
indexed by {1, 2, , ... , m}. For example, the board of Figure 1 contains 76 sequences of
consecutive three holes, which means m = 76. We'll define each jump by its three consecutive holes 
$$Jump= [current\_Node\_Index, intermidate\_Node\_Index, destination\_Node\_Index]$$
We index the jumps as follow:
<br\>
J= [19 Right jumps, 19 Left jumps, 19 Up jumps, 19 Down Jumps]


Given a peg solitaire board, we define a n x m matrix $ A = (a_{ij})$ whose rows and columns
are indexed by holes and jumps respectively by :

$$
a_{ij} = \left\{
    \begin{array}{ll}
        1 & \mbox{if} &\text{ (a peg on the hole i is removed by the jump j)} \\
        -1 & \mbox{if} & \text{ (a peg is placed on the hole i by the jump j)}\\
        0 &  \mbox{otherwise}&
    \end{array}
\right.
$$
In this case, we can represent A as the following:


A = [ A1 | A2 | A3 | A4 ] where Ai is a 33 x 19 matrix

For any binary vector ***p***, #p denotes the number of 1-s in the elements of ***p*** . We
denote ***#ps − #pf*** by $l$. If the given peg solitaire problem is feasible, any feasible
sequence consists of $l$ jumps, because at each jump, the number of pegs decreases by 1.
For example, the peg solitaire problem defined by Figures 1 and 2 is feasible and it
exists a feasible sequence whose length is $ l = 32$ .

Since each jump corresponds to a unit vector of length m, a feasible sequence corresponds to a sequence of $l$ unit vectors $(x^1, x^2, ... , x^l)$ such that $x^k = (x^k_1, x^k_2, ... x^k_m)$ for all $k\in\{1,2, ... , l\}$ and:   
$$
x^k_j = \left\{
    \begin{array}{ll}
        1 & \mbox{if} &\text{ (the k-th move is the jump j)} \\
        0 & \mbox{if} & \text{ (the k-th move is not the jump j)}
    \end{array}
\right.
$$

If a configuration ***p'*** is obtained by applying the jump ***j*** to a configuration ***p*** then
we have: $p' = p − Au$ , where u is the j-th unit vector in $\{0,1\}^m$

From the above discussion we can formulate the peg solitaire problem as the following integer programming problem:
\begin{alignat}{5}
& IP1 &   find           &            &(x^1, x^2, ... , x^l)                       &                              & &\\
&&\text{subject to} & \qquad     & A(x^1+ x^2 + ... + x^l) =p_s -p_f          &                               &(1)&\\
&&                  &            & 0 \le p_s - A(x^1+ x^2 + ... + x^k) \le 1  &(\forall k \in \{1,2,...,l\})& (2)&\\
&&                  &            & x^k_1+ x^k_2+ ... +x^k_m = 1               &(\forall k \in \{1,2,...,l\}) & (3)&\\
&&                  &            &  x^k \in \{0,1\}^m                          &(\forall k \in \{1,2,...,l\})&(4)&\\
\end{alignat}

 * ***Resolution using Python***
 We first concatenate the unknown vectors $(x^1, x^2, ... , x^l)$  into a single vector, we join jumps one after the other, and the vector we are trying to find becomes:
 $$x = [x^1_1, x^1_2, ... x^l_m, | x^2_1, x^2_2, ... x^2_m,| ............. | ,x^l_1, x^l_2, ... x^l_m ] $$
 Then, we use the package **cvxopt** to find a solution to the problem, in the following lines of code we define the cost matrix, the equality and inequality constraints matrices.

In [1]:
import random
import copy
import numpy as np
from time import time
import cvxopt
import cvxopt.glpk
from cvxopt import matrix

In [2]:
def D2toD1(i,j): #This function takes as inputs the coordinates of a hole in the peg solitaire board,
    #and gives the index of the hole in {0,1,...,n-1 =18} as presented in Figure 2
    if i<2 and  1<j<5:
        return( j-2 + i*3)
    elif 1<i<5:
        return( 6+ j + 7*(i-2))
    elif 4<i and  1<j<5:
        return 27 + j-2 + (i-5)*3
    else:
        raise Exception('incorrect coordinates')
        

In [3]:
def D1toD2(l):#This function takes as input the index of the hole, and returns the coordinates of the hole on the peg solitaire 
    # board
    if l<6:
        i=l//3
        j=2+l-i*3
        return(i,j)
    elif 5<l<27:
        i=2+(l-6)//7
        j=l-6-7*(i-2)
        return(i,j)
    elif 26<l<33:
        i=5+(l-27)//3
        j=2+l-27-(i-5)*3
        return(i,j)
    else:
        raise Exception('The index must be between 0 and 32')

In [4]:
def JumptoOrder(jump):# This function takes as inputs the jump=[current_Node_Index, intermidate_Node_Index, destination_Node_Index]
    #and returns the order of the jump in {0,1,....,m-1= 75}.
    if D1toD2(jump[0])[0] == D1toD2(jump[1])[0] == D1toD2(jump[2])[0]:
        if D1toD2(jump[0])[1] == D1toD2(jump[1])[1]-1 == D1toD2(jump[2])[1]-2:
            if jump[0]<6:
                return(D1toD2(jump[0])[0])
            elif 5<jump[0]<26:
                return(2+ 5*(D1toD2(jump[0])[0]-2) +D1toD2(jump[0])[1]  )
            elif 26<jump[0]<33:
                return(17+D1toD2(jump[0])[0]-5)
            else:
                raise Exception('The index of the jump are incorrect')
        if D1toD2(jump[0])[1] == D1toD2(jump[1])[1]+1 == D1toD2(jump[2])[1]+2:
            if jump[0]<6:
                return(D1toD2(jump[0])[0] + 19)
            elif 5<jump[0]<26:
                return(19 + 2+ 5*(D1toD2(jump[0])[0]-2) +( D1toD2(jump[0])[1]-2)  )
            elif 26<jump[0]<33:
                return(19 + 17+D1toD2(jump[0])[0]-5)
            else:
                raise Exception('The index of the jump are incorrect')
    elif D1toD2(jump[0])[1] == D1toD2(jump[1])[1] == D1toD2(jump[2])[1]:
        if D1toD2(jump[0])[0] == D1toD2(jump[1])[0]+1 == D1toD2(jump[2])[0]+2:
            if D1toD2(jump[0])[1]<2:
                return(D1toD2(jump[0])[1] + 2*19)
            elif 1<D1toD2(jump[0])[1]<5:
                return(2*19 + 2+ 5*(D1toD2(jump[0])[1]-2) +( D1toD2(jump[0])[0]-2)  )
            elif 4<D1toD2(jump[0])[1]<7:
                return(2*19 + 17+D1toD2(jump[0])[1]-5)
            else:
                raise Exception('The index of the jump are incorrect')
        if D1toD2(jump[0])[0] == D1toD2(jump[1])[0]-1 == D1toD2(jump[2])[0]-2:
            if D1toD2(jump[0])[1]<2:
                return(D1toD2(jump[0])[1] + 3*19)
            elif 1<D1toD2(jump[0])[1]<5:
                return(3*19 + 2+ 5*(D1toD2(jump[0])[1]-2) + D1toD2(jump[0])[0]  )
            elif 4<D1toD2(jump[0])[1]<7:
                return(3*19 + 17+D1toD2(jump[0])[1]-5)
            else:
                raise Exception('The index of the jump are incorrect')
        else:
            raise Exception('The index of the jump are incorrect')

In [5]:
def OrdertoJump(l): # This function takes as input the index of the jump l in {0,1,...,m-1= 75} and returns the indexes of 
    #the three holes involved in the jump; jump=[current_Node_Index, intermidate_Node_Index, destination_Node_Index].
    if l//19==0:
        if l==0:
            return(0,1,2)
        elif l==1:
            return(3,4,5)
        elif l==17:
            return(27,28,29)
        elif l==18:
            return(30,31,32)
        else:
            i=(l-2)//5 +2
            j=l-2-5*(i-2)
            return(D2toD1(i,j),D2toD1(i,j)+1, D2toD1(i,j)+2)
    if l//19==1:
        if l-19==0:
            return(2,1,0)
        elif l-19==1:
            return(5,4,3)
        elif l-19==17:
            return(29,28,27)
        elif l-19==18:
            return(32,31,30)
        else:
            i=(l-19-2)//5 +2
            j=l-19-2-5*(i-2)  +2
            return(D2toD1(i,j),D2toD1(i,j)-1, D2toD1(i,j)-2)
    if l//19==2:
        if l-2*19==0:
            return(20,13,6)
        elif l-2*19==1:
            return(21,14,7)
        elif l-2*19==17:
            return(25,18,11)
        elif l-2*19==18:
            return(26,19,12)
        else:
            j= (l-2*19-2)//5 +2
            i= l-2*19 -2 -5*(j-2) +2
            return(D2toD1(i,j),D2toD1(i-1,j), D2toD1(i-2,j))
    if l//19==3:
        if l-3*19==0:
            return(6,13,20)
        elif l-3*19==1:
            return(7,14,21)
        elif l-3*19==17:
            return(11,18,25)
        elif l-3*19==18:
            return(12,19,26)
        else:
            j= (l-3*19-2)//5 +2
            i= l -3*19 -2 -5*(j-2)
            return(D2toD1(i,j),D2toD1(i+1,j), D2toD1(i+2,j))

In [6]:
def legal(p):# This function verifies if the vector p contains only 0,1 or not.
    for i in range(len(p)):
        if p[i]<0 or p[i]>1:
            return(False)
    return(True)

In [7]:
n=33 #The number of the nodes in the peg solitaire board
m=76 #The number of jumps
##The matrix A as defined above:
A=np.zeros((n,m))
for i in range(m):
    A[OrdertoJump(i),i]=[1,1,-1]
##The starting configuration
ps=np.ones((n,1))
ps[16]=0
##The finishing configuration
pf=np.zeros((n,1))
pf[16]=1
##number of jumps
l=int(np.sum(ps)-np.sum(pf))
##Equality constraints
##Constraints (1) from the above formulation
B=np.hstack((np.eye(m) for i in range(l)))
E1=np.dot(A,B)
e1=ps-pf
##Constraints (3) from the above formulation
E2=np.zeros((l,l*m))
for i in range(l):
    E2[i,i*m:(i+1)*m]=1
e2=np.ones((l,1))
E=np.vstack((E1,E2))
e=np.vstack((e1,e2))
E=matrix(E, tc='d')
e=matrix(e, tc='d')

In [8]:
##Inequality constraints
##The left side of the (2) inquality constraints
B=np.zeros((m,l*m))
B[:,:m]=np.eye(m)
I1=np.dot(A,B)
for i in range(1,l):
    B=np.zeros((m,l*m))
    B[:,:(i+1)*m]=np.hstack((np.eye(m) for j in range(i+1)))
    I1=np.vstack((I1,np.dot(A,B)))    
i1=np.vstack((ps for i in range(l)))

##The right side of the (2) inquality constraints
B=np.zeros((m,l*m))
B[:,:m]=np.eye(m)
I2=np.dot(-A,B)
for i in range(1,l):
    B=np.zeros((m,l*m))
    B[:,:(i+1)*m]=np.hstack((np.eye(m) for j in range(i+1)))
    I2=np.vstack((I2,np.dot(-A,B)))    
i2=np.vstack((1-ps for i in range(l)))
I=np.vstack((I1,I2))
i=np.vstack((i1,i2))
I=matrix(I, tc='d')
i=matrix(i, tc='d')
cost_matrix=matrix(np.zeros(l*m), tc='d') #There is no objective function to manimize
B=set(range(l*m)) #All the variables are binary

* ***Validation***
<br\ >To verify the correctness of my matrices, I've looked for an example of a solution of the English peg solitaire **fsol**, and I verified that $(E * fsol = e)$ and $(I * fsol \le i)$ are both True, then I consider this test as a valisation of the correcteness of my formulation of the problem.

In [9]:
Jumps=np.array([[14,15,16],[27,22,15],[20,21,22],[23,22,21],[25,24,23],[32,29,24],[17,24,29],[30,31,32],[32,29,24],[8,15,22],
                [0,3,8],[5,10,17],[17,24,29],[29,28,27],[27,22,15],[15,8,3],[6,13,20],[20,21,22],[22,23,24],[12,11,10],[9,10,11]
               ,[26,19,12],[12,11,10],[2,1,0],[0,3,8],[7,8,9],[9,10,11],[11,18,25],[25,24,23],[23,16,9],[4,9,16]])

solution=np.zeros((l,m))

for i in range(l):
    solution[i,JumptoOrder(Jumps[i])]=1

solution=solution.reshape(l*m,1)

print("This solution verifies the 'IP1' equality constraints :" ,sum(np.dot(E,solution)==e)==l+n)

print("This solution verifies the 'IP1' inequality constraints :",sum(np.dot(I,solution)<=i)== 2*n*l)

This solution verifies the 'IP1' equality constraints : [ True]
This solution verifies the 'IP1' inequality constraints : [ True]


In [10]:
?cvxopt.glpk.ilp

In [11]:
# Solving the problem

#(status, optimal_x) = cvxopt.glpk.ilp(c=cost_matrix,G=I,h=i,A=E,b=e,B=B)

 * ***Discussion***
 <br\>The above problem has a solution if and only if the given peg solitaire problem is feasible. Clearly, anysolution $(x^1, x^2, ... , x^l)$ of the integer programming problem corresponds to a feasible sequence of jumps.
<br\> In our formulation of the peg solitaire problem, the number of variables is $m\times l =76*31=2356$ , the number of equality constraints is $n+l= 33+31= 64$, and the number of inequality constraints is $2 \times l \times n = 2 *32*33 = 2046$ .Thus, the size of the integer programming problem is huge and so it is hard to solve the problem by **cvxopt**. In my case, the algorithm kept looking for a solution during 4 hours before I stopped it
<br\> Hence, we use another method of solving the peg solitaire which a combination of backtrack search and pruning techniques based on the above integer programming problem.

## 3 - Backtracking search

In this section, we'll implement a backtrack search algorithm:
* We start from a starting configuration, and we make a legal Jump.
* We consider then the new configuration as our new starting configuration , and we keep recursively searching for a solution.
* If the problem is feasible, we eventually arrive to the finishing configuration, otherwise the problem is unfeasible.

We also use three pruning techniques to improve the efficiency of our backtrack search algorithm:
- **The Upper bound of the number of jumps**
- **The pagoda function**
- **Symmetry**
<img src="img/3.png" alt="Drawing" style="width: 700px;"/>

* ***Upper Bound of the Number of  Jumps***
<br\>**This section has been extracted and detailed from an original paper [1]**
<br\>In this section, we propose a method for finding an upper bound of the number of jumps for each jump j contained in a feasible sequence. In addition, this method is proved to be a very strong tool to check the feasibilities of the given problems.
<br\> We consider the following integer programming problem for each jump j;
\begin{alignat}{5}
& UBj &   \max           &            & x^1_j +  x^2_j + ... + x^l_j                       &                              & &\\
&     &\text{subject to} & \qquad     & A(x^1+ x^2 + ... + x^l) =p_s -p_f          &                               &(1)&\\
&     &                  &            & 0 \le p_s - A(x^1+ x^2 + ... + x^k) \le 1  &(\forall k \in \{1,2,...,l\})& (2)&\\
&     &                  &            & x^k_1+ x^k_2+ ... +x^k_m = 1               &(\forall k \in \{1,2,...,l\}) & (3)&\\
&     &                  &            &  x^k \in \{0,1\}^m                          &(\forall k \in \{1,2,...,l\})&(4)&\\
\end{alignat}

Since the set of constraints of UBj is the same as the constarints of the peg solitaire integer programming problem, the given peg solitaire problem is feasible, if and only if UBj has an optimal solution. Furthermore, if $z_j$ is a solution of the UBj, we are sure that there's no feasible solution for IP1 that contains more than $z_j$ Jumps j .

However, the size of the above problem is equivalent to the original problem IP1 and so, it is hard to solve. In the following, we relax the above problem to a well-solvable problem: 
* We replace $x^1_j +  x^2_j + ... + x^l_j $ by $x_j$ for each $j \in \{1,2, ... , m\}$.
* We drop the constraints (2), (3) and (4) seen above.
<br\> Then we have the following relaxed problem of UBj :
\begin{alignat}{5}
& RUBj &   \max           &            & x_j                    & &                             & &\\
&     &\text{subject to} & \qquad     & Ax =p_s -p_f          &    &                           &(1)&\\
&     &                  &            &  x_j \in  \mathbb{N}    & \qquad                &(\forall j \in \{1,2,...,m\})&(4)&\\
\end{alignat}


* if a problem RUBj is unfeasible for an index $j \in \{1,2,...,m\}$, the original peg solitaire problem is unfeasible and the problem RUBj is also unfeasible for each index j.
* The above problem is a relaxed problem of UBj, the optimal value is an upper bound of the optimal value of UBj.
* The feasibility of RUBj does not guarantee the feasibility of IP1

***Resolution Using Python***

In [12]:
# Defining matrices for RUB integer program
#Equality constraints
Erub = matrix(A, tc='d')
#Inequality constraints : non-negative integers    
Irub = -1*np.eye(m)
Irub = matrix(Irub, tc='d')

irub = np.zeros((m,1))
irub = matrix(irub, tc='d')

Srub = set(range(m))
def solveRUB(jump, starting_config, finishing_config):
    cost = np.zeros(m)
    cost[jump] = -1
    cost = matrix(cost, tc='d')
    
    erub = matrix((starting_config - finishing_config).reshape((33,1)), tc='d')
    
    (status, optimal_x) = cvxopt.glpk.ilp(c=cost,G=Irub,h=irub,A=Erub,b=erub,I=Srub)
    
    if optimal_x:
        x_jump = np.matmul(cost.T, optimal_x)
        x_jump = x_jump[0][0]
        return(-1*x_jump)
    else:
        return None

In [13]:
# We create a list RUB_solution containing the upper bound of all 76 jumps. 
RUB_solution=np.zeros(m)
for k in range(m):
    RUB_solution[k]=solveRUB(k,ps,pf)
RUB_solution

array([2., 1., 3., 4., 3., 4., 1., 1., 2., 1., 1., 1., 3., 4., 3., 4., 1.,
       1., 2., 2., 1., 1., 4., 3., 4., 3., 1., 1., 1., 2., 1., 1., 4., 3.,
       4., 3., 1., 2., 2., 1., 1., 4., 3., 4., 3., 1., 1., 1., 2., 1., 1.,
       4., 3., 4., 3., 1., 2., 2., 1., 3., 4., 3., 4., 1., 1., 2., 1., 1.,
       1., 3., 4., 3., 4., 1., 1., 2.])

We can see that there are many jumps not to be done twice and not to be done three times and so on , So we can prune many brunches during the backtrack searching.

* ***Pagoda Function***
<br\>**This section has been extracted and detailed from original papers [1] and [2]**
<br\>A Pagoda funtion is a function defined on the set of holes $\{1,2,...,n \} \rightarrow  \mathbb{R}$ that satisfies the properties that for every (vertically or horizontally) consecutive three holes $(i_1, i_2, i_3)$, the pagoda function values  $\{pag(i_1),pag(i_2),pag(i_3)\}$ satify $ pag(i_1) +pag(i_2) 	\geqslant pag(i_3)$ , (Clearly, the sequence $(i_1, i_2, i_3)$ is also a consecutive three holes, and so the inequlity $ pag(i_3) +pag(i_2) 	\geqslant pag(i_1)$ also holds.), A Pagoda function corresponds to an assignment of real values to holes on the board satisfying the above properties. 
<br\>For any configuration $p \in \{0,1\}^n$, we denote $$pag(p)= \sum_{i=1}^n pag(i)* p_i $$
<br\> The definition of the pagoda functions implies that if a configuration **p'** is obtained by applying a jump to a configuration **p**, then $peg(p)\geq peg(p')$
<br\>Thus, no intermediate configuration can have $peg(p)\leq peg(p_f)$ for any pagoda funtion . Otherwise we stop extending the partial solution looking for the final configuration.
<br\>Thus, if a given peg solitaire problem is feasible, then the inequality $peg(p)\geq peg(p_f)$ holds for all intermediate configuration and for any pagoda function pag(.), so the existence of a pagoda function pag(.) satisfying $pag(p)< peg(p_f)$ shows that the given peg solitaire problem is unfeasible.
<br\> In **paper [3]** , three pagoda functions are advised for the English peg solitaire to detect dead ends early in the backtracking search. Each of the functions is sparsely populated with non-zero values, increasing the likelihood of breaking the Pagoda condition and so pruning the search.
<img src="img/4.png" alt="Drawing" style="width: 600px;"/>

***Resolution Using Python***

In [14]:
p1=np.array([1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,1])
p2=np.array([0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0])
p3=np.array([0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0])
def pagoda(p):
    return(np.dot(np.transpose(p),p1),np.dot(np.transpose(p),p2),np.dot(np.transpose(p),p2))
    

* ***Symmetry in Peg Solitaire***
<br\>**This section has been extracted and detailed from an original paper [2]**

Peg Solitaire is highly symmetric, which is a significant factor in the difficulty of the problem. This section discusses these symmetries and methods used to break them.
<br\>The english solitaire board has rotational and reflectional symmetry. When composed, these two types of symmetry produce eight symmetries including the identity. That is, a rotation about 0, 90, 180 or 270 degrees with of without a reflection across the horizontal or vertical axes. If one board state can be transformed into another via the application of one of the eight symmetries then the two states are symmetrical, so each configuration has 7 symmetrical configuration that lead to the same end configuration, This can lead to a large amount of wasted effort.
<br\> To break this symmetry, we'll create a **hash table** where we store all the configurations that we have already been through and their symmetries as well. Hence, if we bump into an already stored configuration, there is no need to continue exploring it.

<img src="img/5.png" alt="Drawing" style="width: 350px;"/>
***Resolution Using Python***

In [24]:
def sym1(i,j):# Symmetry about axis (1)
    return(i,6-j)
def sym2(i,j):# Symmetry about axis (2)
    return(6-i,j)
def sym3(i,j):# Symmetry about axis (3)
    return(j,i)
def sym4(i,j):# Symmetry about axis (4)
    return(6-j,6-i)
def sym5(i,j):### sym1 o sym3 (composition of two symmetries)
    return(j,6-i)
def sym6(i,j):### sym1 o sym4
    return(6-j,i)
def sym7(i,j):### sym2 o sym1
    return(6-i,6-j)
def allsym(p):#this function returns all the 8 symmetry configurations of a given configuration p
    q=np.zeros((len(p),7))
    for l in range(len(p)):
        (i,j)=D1toD2(l)
        l1= D2toD1(sym1(i,j)[0],sym1(i,j)[1])
        l2= D2toD1(sym2(i,j)[0],sym2(i,j)[1])
        l3= D2toD1(sym3(i,j)[0],sym3(i,j)[1])
        l4= D2toD1(sym4(i,j)[0],sym4(i,j)[1])
        l5= D2toD1(sym5(i,j)[0],sym5(i,j)[1])
        l6= D2toD1(sym6(i,j)[0],sym6(i,j)[1])
        l7= D2toD1(sym7(i,j)[0],sym7(i,j)[1])
        q[l1,0]=p[l]
        q[l2,1]=p[l]
        q[l3,2]=p[l]
        q[l4,3]=p[l]
        q[l5,4]=p[l]
        q[l6,5]=p[l]
        q[l7,6]=p[l]
    return(q)  

In order to randomly explore jumps among the 76 possible ones, we use the function **numpy.radom.shuffle** to shuffle the columns of A. And we use **numpy.random.seed(seed=4)** to always produce the same columns order.


In [16]:
np.random.seed(seed=4)
s = np.arange(A.shape[1])
np.random.shuffle(s)
A1= np.copy(A)
A1 = A1[:,s]
s


array([49, 66, 60, 48, 14, 26, 72, 24, 29,  4, 19, 73, 43,  5, 37, 47, 41,
       20, 10, 13,  7,  6, 65, 63, 11, 39, 22, 18, 64, 12, 16, 27, 54, 25,
       35, 17, 31, 34, 15, 59, 28, 45, 33, 61, 40, 32, 23, 51,  8, 30, 75,
        2, 53, 62, 70, 56, 21, 67,  0,  3, 52, 42, 38, 44, 71, 36, 57, 68,
       74, 58,  9, 50,  1, 69, 55, 46])

***Backtrack algorithm with the three pruning techniques using Python***

In [17]:
def search(p,pf, upper_bound, path , dictionary):#This function returns 0 if the problem is unfeasible, and 1 otherwise
    # p: an intermediate configuration from which we try to reach the final configuration
    # pf:The finishing configuration we are trying to reach, this parameter remain the same in all the iteration of the backtrack search
    # upper_bound: a list of the upper bound of all the jumps being updated at each step.
    # path: a list of the jumps being explored.
    # dictionary: a hash table with all the solutions that has been already computed.
    
    rest=int(sum(p)-sum(pf))
    if rest<=0:
        if all(p==pf):
            return(True)
        else: 
            return(False)
    else:
        if pagoda(p)[0] < pagoda(pf)[0] or pagoda(p)[1] < pagoda(pf)[1] or pagoda(p)[2] < pagoda(pf)[2]:
            return (False)
        else:
            for i in range(m):
                x=np.zeros((m,1))
                x[s[i]]=1
                optimal_x = solveRUB(s[i], p, pf)
                if optimal_x is None:
                    return False
                if not legal(p-np.dot(A,x)):
                    continue
                if upper_bound[s[i]]<=0:
                    continue
                upper_bound[s[i]]-=1
                p=p-np.dot(A,x)
                path.append(s[i])

                mytuple =  hash(tuple(p[:,0]))
                
                if mytuple in dictionary.keys():
                    y = dictionary[mytuple]
                else:

                    y = search(p, pf, upper_bound, path, dictionary)
                    dictionary[mytuple] = y
                    q=allsym(p)
                    for j in range(7):
                        mytuple = hash(tuple((q[:,j])))
                        dictionary[mytuple] = y

                if y == 1:
                    return True
                else:
                    upper_bound[s[i]] += 1
                    p = p + np.dot(A,x)
                    path.pop()
    return(False)

In [18]:
def solve_peg_solitaire(ps,pf):#If the problem is feasible, this function returns the path to get to the end configuration
    #as a list of successive jumps, and None otherwise.
    #ps: is the starting configuration of our peg solitaire problem.
    #pf: is the finishing configuration of our peg solitaire problem.
    dictionary={}
    upper_bound=np.zeros(m)
    path=[]
    for k in range(m):
        optimal_x = solveRUB(k,ps,pf)
        if optimal_x is None:
            print("Unfeasible problem")
            return None
        else:
            upper_bound[k] = optimal_x
    if search(ps, pf, upper_bound, path , dictionary)== 0 :
        print("Unfeasible")
        return(None)
    else:
        print(" feasible problem")
        return (path)
    

## 3 - Test

We tested the function **solve_peg_solitaire** on different starting configurations, and in all the cases , the code finds a solution in less than 40s, which is a quite fast backtracking search.

In [32]:
t1=time()
solution_path=solve_peg_solitaire(ps,pf)
t2=time()
print("The code found a solution in:", round(t2-t1,2), "s")
print(solution_path)

 feasible problem
The code found a solution in: 35.07 s
[48, 66, 24, 29, 4, 64, 5, 25, 59, 53, 14, 72, 12, 54, 18, 56, 3, 5, 57, 9, 69, 51, 25, 70, 72, 54, 34, 32, 12, 43, 8]


* ***Validation***
<br\>In the following lines of code, we ensure that the solution returned by the backtracking search algorithm verifies the constraints of the IP1 problem.

In [21]:

print("This solution verifies the 'IP1' equality constraints :",sum(np.dot(E,solution)==e)==l+n)

print("This solution verifies the 'IP1' inequality constraints :", sum(np.dot(I,solution)<=i)== 2*n*l)

This solution verifies the 'IP1' equality constraints : [ True]
This solution verifies the 'IP1' inequality constraints : [ True]


In [22]:
def print_conf(p):# This function prints a given configuration in the form of an English peg solitaire board.
    a = np.empty((7,7), dtype='str')
    a[:]= ' '
    for k in range(len(p)):
        (i,j)=D1toD2(k)
        if p[k]==1:
            a[i,j]= '\u25CF'
        elif p[k]==0:
            a[i,j]= 'o'
    print(a)
def print_path(ps,pf, path):# This function prints all the moves corresponding to jumps in path, in the order from the start
    #configuration until the finishing one.
    p=np.copy(ps)
    print('------Initial Configuration' + '------\n')
    print_conf(ps)
    for i in range(len(path)-1):
        x=np.zeros((m,1))
        x[path[i]]=1
        p= p -np.dot(A,x)
        print('-------------Move ' + str(i+1) +'-------'+ 'jump  ' +  str(D1toD2(OrdertoJump(path[i])[0]))
              +str(D1toD2(OrdertoJump(path[i])[1]))+'-->'+str(D1toD2(OrdertoJump(path[i])[2])) +'-------------\n')
        print_conf(p)
    x=np.zeros((m,1))
    x[path[-1]]=1
    p= p -np.dot(A,x)
    if np.array_equal(p,p):
        print('------Final Configuration'+'-------'+ 'jump  ' + str(D1toD2(OrdertoJump(path[-1])[0]))
          +str(D1toD2(OrdertoJump(path[-1])[1]))+'-->'+str(D1toD2(OrdertoJump(path[-1])[2])) + '------\n')
        print_conf(pf)
    

In [23]:
print_path(ps,pf, solution_path)

------Initial Configuration------

[[' ' ' ' '●' '●' '●' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 ['●' '●' '●' '●' '●' '●' '●']
 ['●' '●' '●' 'o' '●' '●' '●']
 ['●' '●' '●' '●' '●' '●' '●']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']]
-------------Move 1-------jump  (5, 3)(4, 3)-->(3, 3)-------------

[[' ' ' ' '●' '●' '●' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 ['●' '●' '●' '●' '●' '●' '●']
 ['●' '●' '●' '●' '●' '●' '●']
 ['●' '●' '●' 'o' '●' '●' '●']
 [' ' ' ' '●' 'o' '●' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']]
-------------Move 2-------jump  (2, 3)(3, 3)-->(4, 3)-------------

[[' ' ' ' '●' '●' '●' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 ['●' '●' '●' 'o' '●' '●' '●']
 ['●' '●' '●' 'o' '●' '●' '●']
 ['●' '●' '●' '●' '●' '●' '●']
 [' ' ' ' '●' 'o' '●' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']]
-------------Move 3-------jump  (2, 5)(2, 4)-->(2, 3)-------------

[[' ' ' ' '●' '●' '●' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 ['●' '●' '●' '●' 'o' 'o' '●']
 ['●' '●' '●' 

<br\>**The following examples has been extracted from an original paper [3] and solved by our backtrack searching algorithm.**
<img src="img/6.png" alt="Drawing" style="width: 600px;"/>

In [33]:
p5=np.zeros((n,1))
p5[[4,8,9,10,16,23],0]=1
print_conf(p5)
t1=time()
solution_path1=solve_peg_solitaire(p5,pf)
t2=time()
print("The code found a solution in:", round(t2-t1,2), "s")
print_path(p5,pf,solution_path1)

[[' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 ['o' 'o' '●' '●' '●' 'o' 'o']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']]
 feasible problem
The code found a solution in: 0.2 s
------Initial Configuration------

[[' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 ['o' 'o' '●' '●' '●' 'o' 'o']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']]
-------------Move 1-------jump  (2, 3)(2, 4)-->(2, 5)-------------

[[' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 ['o' 'o' '●' 'o' 'o' '●' 'o']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']]
-------------Move 2-------jump  (4, 3)(3, 3)-->(2, 3)-------------

[[' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 ['o' 'o' '●' '●' 'o' '●' 'o']
 ['o' 'o' 'o' 'o' 'o' 'o' 'o

In [30]:
p7=np.ones((n,1))
p7[[0,1,2,3,5,6,7,8,10,11,12,13,19,20,21,22,24,25,26,27,29,30,32,31],0]=0
print_conf(p7)
t1=time()
solution_path2=solve_peg_solitaire(p7,pf)
t2= time()
print("The code found a solution in:", round(t1-t2,2), "s")
print_path(p7,pf,solution_path2)

[[' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 ['o' '●' '●' '●' '●' '●' 'o']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']]
 feasible problem
The code found a solution in: -0.52 s
------Initial Configuration------

[[' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 ['o' '●' '●' '●' '●' '●' 'o']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']]
-------------Move 1-------jump  (3, 2)(3, 1)-->(3, 0)-------------

[[' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 ['●' 'o' 'o' '●' '●' '●' 'o']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']]
-------------Move 2-------jump  (3, 4)(3, 3)-->(3, 2)-------------

[[' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 ['●' 'o' '●' 'o' 'o' '●' 

In [34]:
p6=np.zeros((n,1))
p6[[0,1,2,3,4,5,8,9,10,15,17],0]=1
print_conf(p6)
t1=time()
solution_path2=solve_peg_solitaire(p6,pf)
t2=time()
print("The code found a solution in:", round(t2-t1,2), "s")
print_path(p6,pf,solution_path2)

[[' ' ' ' '●' '●' '●' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 ['o' 'o' '●' '●' '●' 'o' 'o']
 ['o' 'o' '●' 'o' '●' 'o' 'o']
 ['o' 'o' 'o' 'o' 'o' 'o' 'o']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']]
 feasible problem
The code found a solution in: 2.77 s
------Initial Configuration------

[[' ' ' ' '●' '●' '●' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 ['o' 'o' '●' '●' '●' 'o' 'o']
 ['o' 'o' '●' 'o' '●' 'o' 'o']
 ['o' 'o' 'o' 'o' 'o' 'o' 'o']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']]
-------------Move 1-------jump  (2, 3)(2, 4)-->(2, 5)-------------

[[' ' ' ' '●' '●' '●' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 ['o' 'o' '●' 'o' 'o' '●' 'o']
 ['o' 'o' '●' 'o' '●' 'o' 'o']
 ['o' 'o' 'o' 'o' 'o' 'o' 'o']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']]
-------------Move 2-------jump  (0, 4)(1, 4)-->(2, 4)-------------

[[' ' ' ' '●' '●' 'o' ' ' ' ']
 [' ' ' ' '●' '●' 'o' ' ' ' ']
 ['o' 'o' '●' 'o' '●' '●' 'o']
 ['o' 'o' '●' 'o' '●' 'o' '

In [35]:
p8=np.zeros((n,1))
p8[[1,3,4,5,7,8,9,10,11,16,23,27,28,29,30,31,32],0]=1
print_conf(p8)
t1=time()
solution_path2=solve_peg_solitaire(p8,pf)
t2=time()
print("The code found a solution in:", round(t2-t1,2), "s")
print_path(p8,pf,solution_path2)

[[' ' ' ' 'o' '●' 'o' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 ['o' '●' '●' '●' '●' '●' 'o']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']]
 feasible problem
The code found a solution in: 24.09 s
------Initial Configuration------

[[' ' ' ' 'o' '●' 'o' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 ['o' '●' '●' '●' '●' '●' 'o']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']]
-------------Move 1-------jump  (1, 2)(2, 2)-->(3, 2)-------------

[[' ' ' ' 'o' '●' 'o' ' ' ' ']
 [' ' ' ' 'o' '●' '●' ' ' ' ']
 ['o' '●' 'o' '●' '●' '●' 'o']
 ['o' 'o' '●' '●' 'o' 'o' 'o']
 ['o' 'o' 'o' '●' 'o' 'o' 'o']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']]
-------------Move 2-------jump  (6, 4)(5, 4)-->(4, 4)-------------

[[' ' ' ' 'o' '●' 'o' ' ' ' ']
 [' ' ' ' 'o' '●' '●' ' ' ' ']
 ['o' '●' 'o' '●' '●' '●' 'o']
 ['o' 'o' '●' '●' 'o' 'o' 

In [36]:
p9=np.zeros((n,1))
p9[[4,8,9,10,14,15,16,17,18,20,21,22,23,24,25,26],0]=1
print_conf(p9)
t1=time()
solution_path2=solve_peg_solitaire(p9,pf)
t2=time()
print("The code found a solution in:", round(t2-t1,2), "s")
print_path(p9,pf,solution_path2)

[[' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 ['o' 'o' '●' '●' '●' 'o' 'o']
 ['o' '●' '●' '●' '●' '●' 'o']
 ['●' '●' '●' '●' '●' '●' '●']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']]
 feasible problem
The code found a solution in: 1.28 s
------Initial Configuration------

[[' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 ['o' 'o' '●' '●' '●' 'o' 'o']
 ['o' '●' '●' '●' '●' '●' 'o']
 ['●' '●' '●' '●' '●' '●' '●']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']]
-------------Move 1-------jump  (3, 2)(3, 1)-->(3, 0)-------------

[[' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 ['o' 'o' '●' '●' '●' 'o' 'o']
 ['●' 'o' 'o' '●' '●' '●' 'o']
 ['●' '●' '●' '●' '●' '●' '●']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' 'o' 'o' ' ' ' ']]
-------------Move 2-------jump  (3, 4)(4, 4)-->(5, 4)-------------

[[' ' ' ' 'o' 'o' 'o' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']
 ['o' 'o' '●' '●' '●' 'o' 'o']
 ['●' 'o' 'o' '●' 'o' '●' '

In [37]:
p10=np.ones((n,1))
p10[[0,2,6,12,16,20,26,30,32],0]=0
print_conf(p10)
t1=time()
solution_path2=solve_peg_solitaire(p10,pf)
t2=time()
print("The code found a solution in:", round(t2-t1,2), "s")
print_path(p10,pf,solution_path2)

[[' ' ' ' 'o' '●' 'o' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 ['o' '●' '●' '●' '●' '●' 'o']
 ['●' '●' '●' 'o' '●' '●' '●']
 ['o' '●' '●' '●' '●' '●' 'o']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']]
 feasible problem
The code found a solution in: 2.65 s
------Initial Configuration------

[[' ' ' ' 'o' '●' 'o' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 ['o' '●' '●' '●' '●' '●' 'o']
 ['●' '●' '●' 'o' '●' '●' '●']
 ['o' '●' '●' '●' '●' '●' 'o']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']]
-------------Move 1-------jump  (5, 3)(4, 3)-->(3, 3)-------------

[[' ' ' ' 'o' '●' 'o' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 ['o' '●' '●' '●' '●' '●' 'o']
 ['●' '●' '●' '●' '●' '●' '●']
 ['o' '●' '●' 'o' '●' '●' 'o']
 [' ' ' ' '●' 'o' '●' ' ' ' ']
 [' ' ' ' 'o' '●' 'o' ' ' ' ']]
-------------Move 2-------jump  (2, 3)(3, 3)-->(4, 3)-------------

[[' ' ' ' 'o' '●' 'o' ' ' ' ']
 [' ' ' ' '●' '●' '●' ' ' ' ']
 ['o' '●' '●' 'o' '●' '●' 'o']
 ['●' '●' '●' 'o' '●' '●' '

# **Conclusion**
The formulation of the Peg Solitaire puzzle as an integer programming problem was very interesting, but the complexity was very high. Thus, it's hard to solve this problem with usual personal computer. Then we implement a backtrack search algorithm with tree pruning techniques (Upper bound of the number of jumps, Pagoda function, a hash table to store all the intermediate configurations and thier symmetrical configurations), we ended up solving the peg solitaire puzzle in very reasonable time (less than 40s).
<br\>Finally, I have learned in this project a lot of methods and tips that could be very useful in solving other discrete optimization problems.

***Bibliography***

[1] -  Masashi Kiyomi and Tomomi Matsui. **Integer programming based algorithms for peg solitaire problems.** ,2001 , available on :https://hugues-talbot.github.io/files/Peg_Solitaire_2.pdf
<br\>[2] -  Christopher Jefferson, Angela Miguel, Ian Miguel, **Modelling and Solving English Peg Solitaire**, available on : https://hugues-talbot.github.io/files/Peg_Solitaire_1.pdf
<br\>[3] -  Armando B. Matos. **Trying to solve Peg Solitaire by a simple, depth-first search**. 2002, available on :http://www.dcc.fc.up.pt/~acm/tpeg.pdf
<br\>[4] -  Course Discrete Optimisation, Hugues Talbot, Centrale Supelec, CVN, 2019, link of the course: https://hugues-talbot.github.io/teaching/2018_Discrete_Optimisation_CS